In [73]:
from pathlib import Path
import pandas as pd
from sklearn.linear_model import LinearRegression
import hvplot.pandas

In [105]:
us_suicides_total = pd.read_csv(
    r"../Resources/Clean/us_suicides_1985-2021.csv"
)
russia_suicides_total = pd.read_csv(
    r"../Resources/Clean/russia_suicides_1985-2016.csv"
)
japan_suicides_total = pd.read_csv(
    r"../Resources/Clean/japan_suicides_1985-2016.csv"
)
canada_suicides_total = pd.read_csv(
    r"../Resources/Clean/canada_suicides_1985-2016.csv"
)
germany_suicides_total = pd.read_csv(
    r"../Resources/Clean/germany_suicides_1985-2016.csv"
)
untreated_mental_illness = pd.read_csv(
    r"../Resources/Clean/untreated_mental_illness_total_2018-2019.csv"
).assign(Year="2018-2019")
metrics_988 = pd.read_csv(
    r"../Resources/Clean/988_performance_metrics.csv"
)

In [75]:
untreated_mental_illness.set_index("Year", inplace=True)

In [76]:
untreated_mental_illness.hvplot.bar(ylim=(0, 100))

:Bars   [Year,Variable]   (value)

In [77]:
us_suicides_total.hvplot(x="Year", y="Number of Suicides", rot=45).opts(
    height = 450, width = 800
) * us_suicides_total.hvplot.scatter(x="Year", y="Number of Suicides", rot=45, c="k").opts(
    height = 450, width = 800
)

:Overlay
   .Curve.I   :Curve   [Year]   (Number of Suicides)
   .Scatter.I :Scatter   [Year]   (Number of Suicides)

In [104]:
russia_suicides_total.hvplot(x="Year", y="Number of Suicides", rot=45).opts(
    height = 450, width = 800
) * russia_suicides_total.hvplot.scatter(x="Year", y="Number of Suicides", rot=45, c="k").opts(
    height = 450, width = 800
) * us_suicides_total.hvplot(x="Year", y="Number of Suicides", rot=45).opts(
    height = 450, width = 800
) * us_suicides_total.hvplot.scatter(x="Year", y="Number of Suicides", rot=45, c="k").opts(
    height = 450, width = 800
) * japan_suicides_total.hvplot(x="Year", y="Number of Suicides", rot=45).opts(
    height = 450, width = 800
) * japan_suicides_total.hvplot.scatter(x="Year", y="Number of Suicides", rot=45, c="k").opts(
    height = 450, width = 800
) * canada_suicides_total.hvplot(x="Year", y="Number of Suicides", rot=45).opts(
    height = 450, width = 800
) * canada_suicides_total.hvplot.scatter(x="Year", y="Number of Suicides", rot=45, c="k").opts(
    height = 450, width = 800
) * germany_suicides_total.hvplot(x="Year", y="Number of Suicides", rot=45).opts(
    height = 450, width = 800
) * germany_suicides_total.hvplot.scatter(x="Year", y="Number of Suicides", rot=45, c="k").opts(
    height = 450, width = 800
)

:Overlay
   .Curve.I     :Curve   [Year]   (Number of Suicides)
   .Scatter.I   :Scatter   [Year]   (Number of Suicides)
   .Curve.II    :Curve   [Year]   (Number of Suicides)
   .Scatter.II  :Scatter   [Year]   (Number of Suicides)
   .Curve.III   :Curve   [Year]   (Number of Suicides)
   .Scatter.III :Scatter   [Year]   (Number of Suicides)
   .Curve.IV    :Curve   [Year]   (Number of Suicides)
   .Scatter.IV  :Scatter   [Year]   (Number of Suicides)
   .Curve.V     :Curve   [Year]   (Number of Suicides)
   .Scatter.V   :Scatter   [Year]   (Number of Suicides)

In [87]:
japan_suicides_total.hvplot(x="Year", y="Number of Suicides", rot=45).opts(
    height = 450, width = 800
) * japan_suicides_total.hvplot.scatter(x="Year", y="Number of Suicides", rot=45, c="k").opts(
    height = 450, width = 800
)

:Overlay
   .Curve.I   :Curve   [Year]   (Number of Suicides)
   .Scatter.I :Scatter   [Year]   (Number of Suicides)

In [80]:
# Separate the features (Year) and target (Number of Suicides)
X = us_suicides_total[['Year']]
y = us_suicides_total['Number of Suicides']

# Create and fit a linear regression model
model = LinearRegression()
model.fit(X, y)

# Generate predictions using the fitted model
y_pred = model.predict(X)

# Create scatter plots using hvPlot
scatter_actual = us_suicides_total.hvplot.scatter(x='Year', y='Number of Suicides', rot=37)
scatter_predicted = pd.DataFrame({'Year': us_suicides_total['Year'], 'Predicted Suicides': y_pred}).hvplot.line(x='Year', y='Predicted Suicides', color="red")

# Combine the scatter plots
scatter_actual * scatter_predicted

:Overlay
   .Scatter.I :Scatter   [Year]   (Number of Suicides)
   .Curve.I   :Curve   [Year]   (Predicted Suicides)

In [117]:
metrics_988 = metrics_988.rename(columns={"Unnamed: 0":"Month"})
metrics_988 = metrics_988.iloc[::-1]

In [125]:
metrics_988.hvplot(x = "Month", y = "Total Routed", rot = 45).opts(height = 450, width = 800, yformatter="%.0f") + metrics_988.hvplot(x = "Month", y = "Routed Texts", rot = 45).opts(height = 450, width = 800, yformatter="%.0f")

:Layout
   .Curve.I  :Curve   [Month]   (Total Routed)
   .Curve.II :Curve   [Month]   (Routed Texts)